Data: 
Option prices and implied volatility from post-no-preference option chain dataset. data spans from 2019-current. collected more recently on mon-wed-fri. Many but not all options are posted there for each security.

AAPL price data from Kaggle(?)

Treasury Bond rates from home.treasury.gov. Daily treasury par yield rates

In [46]:
import pandas as pd
import numpy as np
#10m rows takes about 30 seconds... Expect long processing times for full data. do in batches.
#f10m = pd.read_csv('pnp_options.csv', nrows=10000000)
#print(sorted(set(f10m['act_symbol'])))
#print(len(set(f10m['act_symbol'])))

f1m = pd.read_csv('pnp_options.csv', usecols=['date', 'act_symbol', 'expiration', 'strike', 'call_put', 'bid', 'ask',
       'vol'] ,nrows = 1000000)

prices = pd.read_csv('HistoricalQuotes.csv')

rates = pd.read_csv('treasury_2019.csv', usecols=['Date', '1 Mo', '2 Mo', '3 Mo', '6 Mo', '1 Yr'])


### NOTE...

The following data engineering **assumes**:

1. we are only interested in AAPL options. if we want to expand, we can simply modify the first line to include more act_symbol values

2. our prices dataframe includes prices covering the entire range of dates for option prices, plus an extra n_prices before the earliest option price, such that we can utilize the n_prices preceding the option pricing in our LSTM model down the line 

3. our rates dataframe also contains treasury rates with dates covering all issuance dates of options.
    
4. all of our option time-to-expiries are closest to 1-3 months, not 6+ months. in the 1m row data the longest time-to-expiry is 62 days. if we see in the full data a time-to-expiry longer than 135 days, we need to add an option to use the 6 month treasury rate in our determine_r function.


**Also**, you will probably want to split this cell into multiple smaller ones when we're working with the full data. some of the actions might be computationally expensive.

In [117]:
df = f1m[f1m['act_symbol'] == 'AAPL']
aapl_prices = prices.copy()
rate = rates.copy()

#remove expiration date, replace with int # of days until expiration
df['days_expiry'] = (pd.to_datetime(df['expiration']) - pd.to_datetime(df['date'])).dt.days
df = df.drop(['expiration'], axis=1)

#format dates on df, aapl_prices, rates to match each other

df['date'] = pd.to_numeric(df['date'].str.replace('-','')) #shift date from sat to fri, to match treasury rate dates
aapl_prices['Date'] = pd.to_datetime(aapl_prices['Date'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')
aapl_prices['Date'] = pd.to_numeric(aapl_prices['Date'].str.replace('-',''))
rate['Date'] = pd.to_datetime(rate['Date'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')
rate['Date'] = pd.to_numeric(rate['Date'].str.replace('-',''))

#merge rate with df on date, using forward/backwards fill approach, as some dates of options and treasury prices do not line up...
#reindex rate using date vals from
rate_reindexed = rate.set_index('Date').reindex(df['date'])
rate_reindexed = rate_reindexed.ffill().bfill()
df = pd.merge(df, rate_reindexed, left_on='date', right_index=True, how='left')

#choose risk free rate 'r', based on which treasury rate matures closest to the expiration date of the option.
#Then drop other treasury columns leaving just 'r'
def determine_r(row):
    if row['days_expiry'] < 45:
        return row['1 Mo']
    elif 45 <= row['days_expiry'] < 75:
        return row['2 Mo']
    else:
        return row['3 Mo']

df['r'] = df.apply(determine_r, axis=1)
df = df.drop(['1 Mo', '2 Mo', '3 Mo', '6 Mo', '1 Yr'], axis=1)

#Now to the price dataframe...

print(df)

#print(df.sort_values(by='days_expiry', ascending=True))
#convert $ objects in security price df to decimal...
#Decimal(re.sub(['^0-9.'], '', aapl_prices[' Close/Last', 'Open', 'High', 'Low']))
#print(aapl_prices.head()) 


/var/folders/49/x210g1j14bs717pp0q4xf3zr0000gq/T/ipykernel_3596/2621196360.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['days_expiry'] = (pd.to_datetime(df['expiration']) - pd.to_datetime(df['date'])).dt.days


            date act_symbol  strike call_put   bid    ask     vol  \
142     20190209       AAPL   145.0     Call  25.3  26.05  0.4236   
142     20190209       AAPL   145.0     Call  25.3  26.05  0.4236   
142     20190209       AAPL   145.0     Call  25.3  26.05  0.4236   
142     20190209       AAPL   145.0     Call  25.3  26.05  0.4236   
142     20190209       AAPL   145.0     Call  25.3  26.05  0.4236   
...          ...        ...     ...      ...   ...    ...     ...   
970840  20190907       AAPL   245.0      Put  29.9  33.80  0.2110   
970840  20190907       AAPL   245.0      Put  29.9  33.80  0.2110   
970840  20190907       AAPL   245.0      Put  29.9  33.80  0.2110   
970840  20190907       AAPL   245.0      Put  29.9  33.80  0.2110   
970840  20190907       AAPL   245.0      Put  29.9  33.80  0.2110   

        days_expiry     r  
142              13  2.42  
142              13  2.42  
142              13  2.42  
142              13  2.42  
142              13  2.42  
...

In [115]:
#number of price datapoints for each option price instance
n_prices = 20

#formatting dates
apdf = aapl_prices.copy()
df['date'] = pd.to_numeric(df['date'].str.replace('-',''))
apdf['Date'] = pd.to_datetime(apdf['Date'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')
apdf['Date'] = pd.to_numeric(apdf['Date'].str.replace('-',''))

#reducing price df to relevant dates (n_prices dates before the earliest option price, up to the latest option price)
minDate = apdf.index[apdf['Date'] == min(df['date'])-1].tolist()[0]
maxDate = apdf.index[apdf['Date'] == max(df['date'])-1].tolist()[0]
red_prices = apdf.loc[maxDate:(minDate+n_prices),].sort_values(by=['Date'])

print(minDate)
print(apdf.loc[maxDate:(minDate+n_prices),])
#print(apdf[apdf['Date'] == fdf['date'][100] - 1])

265
         Date  Close/Last
120  20190906     $213.26
121  20190905     $213.28
122  20190904     $209.19
123  20190903      $205.7
124  20190830     $208.74
..        ...         ...
281  20190116     $154.94
282  20190115     $153.07
283  20190114        $150
284  20190111     $152.29
285  20190110      $153.8

[166 rows x 2 columns]


/var/folders/49/x210g1j14bs717pp0q4xf3zr0000gq/T/ipykernel_60170/2658561103.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fdf['date'] = pd.to_numeric(fdf['date'].str.replace('-',''))
